# Segmenting and Clustering Notebook

### Create Database using previous notebook

In [2]:
import pandas as pd
import requests
!conda install -c conda-forge beautifulsoup4  --yes  #Install BS4 if environment does not have installed
from bs4 import BeautifulSoup
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,features='html')

#print(soup.prettify())
post_table = soup.find('table',{'class':'wikitable sortable'})

pcode = []
bor = []
neigh = []

rows = post_table.find_all('tr')
    
for row in rows:
    cells = row.find_all('td')
        
        
    if len(cells) > 1:
          
        if cells[1].text.strip()== 'Not assigned':
             next 
        else:
            c0 = cells[0].text.strip()
            pcode.append(c0)
            
            c1 = cells[1].text.strip()
            bor.append(c1)
            
            c2 = cells[2].text.strip()
               
            if c2 == 'Not assigned':    
              neigh.append(c1)
            else:
              neigh.append(c2)
            
wiki = pd.DataFrame({'PostalCode':pcode,'Borough':bor,'Neighbourhood':neigh})
wiki.head(10)

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    numpy-1.18.1               |   py36h95a1406_0         5.2 MB  conda-forge
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    scipy-1.4.1                |   py36h921218d_0        18.9 MB  conda-forg

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Downtown Toronto,Queen's Park
6,M9A,Etobicoke,Islington Avenue
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


## Import geocoder package

### Read coordinates into DataFrame

In [3]:
geo = pd.read_csv('http://cocl.us/Geospatial_data')

### Merge dataframes together on the Postal Code Column as the index had to rename the postal code column from the csv as name was slightly different to that in my wiki dataframe

In [4]:
wiki_t = wiki.merge(geo.rename(columns={"Postal Code":"PostalCode"}),on="PostalCode")

### Check to see if merged correctly

In [5]:
wiki_t.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Heights,43.718518,-79.464763
4,M6A,North York,Lawrence Manor,43.718518,-79.464763


#### Aggregate the Neighbourhoods by Postal Code

In [6]:
wiki_1 = wiki.groupby('PostalCode')['Neighbourhood'].apply(list).to_frame()

#### Get unique Postal Codes while preserving order and convert to dataframes to be merged

In [12]:
pc1 =[]
b1=[]
count = 0
for i in wiki_t['PostalCode']:
    if i not in pc1:
        pc1.append(i)
        b1.append(wiki_t['Borough'][count])
        count = count + 1
        
df1 = pd.DataFrame({'PostalCode':pc1,'Borough':b1})
df2 = wiki_1.merge(geo.rename(columns={"Postal Code":"PostalCode"}),on="PostalCode")
df3 = df1.merge(df2,on="PostalCode")

In [13]:
df3.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,[Parkwoods],43.753259,-79.329656
1,M4A,North York,[Victoria Village],43.725882,-79.315572
2,M5A,Downtown Toronto,[Harbourfront],43.654260,-79.360636
3,M6A,North York,"[Lawrence Heights, Lawrence Manor]",43.718518,-79.464763
4,M7A,North York,[Queen's Park],43.662301,-79.389494
